# Convolutional neural network testing
Use this notebook to fine-tune pre-trained Xception from Keras found here https://keras.io/applications/#xception
NB: More trained models can be found here https://keras.io/applications/

### My own utility functions

In [1]:
import glob
import os
import math
from random import shuffle
import time
import numpy as np
import random
import cv2 as cv
import matplotlib.pyplot as plt
from keras.utils import np_utils

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def get_image_files(root_dir, img_types):
    #os.walk creates 3-tuple with (dirpath, dirnames, filenames)
    
    # Get all the root directories, subdirectories, and files
    full_paths = [x for x in os.walk(root_dir)] 
    imgs_temp = [os.path.join(ds,f) for ds,_,fs in full_paths for f in fs if f]   
    
    # Filter out so only have directories with .jpg, .tiff, .tif, .png, .jpeg
    imgs = [j for j in imgs_temp if any (k in j for k in img_types)]
    return imgs

def get_dimensions(files):
    # Set starting points for min and max dimensions
    min_height, min_width = 10000, 10000
    max_height, max_width = 0, 0
    
    for f in files:
        # Read in images
        img = cv.imread(f) # Read in images
        h,w = img.shape[:2] # get height and width
        
        # Update min and max values, if necessary
        if h < min_height:
            min_height = h 
        if h > max_height:
            max_height = h
        if w < min_width:
            min_width = w
        if w > max_width:
            max_width = w
            
    return min_height, min_width, max_height, max_width

def make_labels(files):
    # Assume input is a list of complete file paths.
    # Count the number of unique directory names that are immediate parent of the files.
    # Order the directory names alphabetically from a-z, and associate labels accordingly.
    set_temp = {x.split('/')[-2] for x in files} #doing as set to get only unique values
    list_temp = list(set_temp) #Change to list so can interate over it
    list_new = sorted(list_temp) #Alphabetizing
    label_dict = {list_new[x]:x for x in range(len(list_new))} #create dictionary with category:index
    
    return label_dict

def make_train_val(files, labels):
    train=[]
    valid = []
    train_labels = []
    valid_labels = []
    train_prop = 0.6 #proportion of data set that will be training
    for key in labels: #going through each key
        temp = [f for f in files if key in f] #getting all files in a specific category (ie key)
        train.extend(temp[:math.ceil(train_prop*len(temp))]) #training data set
        valid.extend(temp[math.ceil(train_prop*len(temp)):]) # validation data set
    train_labels = [x.split('/')[-2] for x in train]
    valid_labels = [x.split('/')[-2] for x in valid]
    return train, valid, train_labels, valid_labels

def make_train_val_test(files, labels):
    train=[]
    valid = []
    test =[]
    train_labels = []
    valid_labels = []
    test_labels = []
    train_prop = 0.6 #proportion of data set that will be training
    val_prop = 0.2 #proprotion of dataset that is validation
    for key in labels: #going through each key
        temp = [f for f in files if key in f] #getting all files in a specific category (ie key)
        lower_prop = math.ceil(train_prop*len(temp))
        train.extend(temp[:lower_prop]) #training data set
        valid.extend(temp[lower_prop:lower_prop+math.ceil(val_prop*len(temp))]) # validation data set
        test.extend(temp[lower_prop+math.ceil(val_prop*len(temp)):])
    train_labels = [x.split('/')[-2] for x in train]
    valid_labels = [x.split('/')[-2] for x in valid]
    test_labels =  [x.split('/')[-2] for x in test]
    return train, valid, test, train_labels, valid_labels, test_labels

## Generator function

In [3]:
def get_batches(files, label_map, batch_size, resize_size, num_color_channels, augment=False, predict=False):
    shuffle(files)
    count = 0
    num_files = len(files)
    num_classes = len(label_map)
    
    batch_out = np.zeros((batch_size, resize_size[0], resize_size[1], num_color_channels), dtype=np.uint8)
    labels_out = np.zeros((batch_size,num_classes)) #one-hot labeling, which is why have num_classes num of col.   

    while True: # while True is to ensure when yielding that start here and not previous lines

        f = files[count]
        img = cv.imread(f)       

        # Resize
        # First resize while keeping aspect ratio
        rows,cols = img.shape[:2] # Define in input num_color_channels in case want black and white
        rc_ratio = rows/cols
        if resize_size[0] > int(resize_size[1]*rc_ratio):# if resize rows > rows with given aspect ratio
            img = cv.resize(img, (resize_size[1], int(resize_size[1]*rc_ratio)))#NB: resize dim arg are col,row
        else:
            img = cv.resize(img, (int(resize_size[0]/rc_ratio), resize_size[0]))
            
        # Second, pad to final size
        rows,cols = img.shape[:2] #find new num rows and col of resized image
        res = np.zeros((resize_size[0], resize_size[1], num_color_channels), dtype=np.uint8)#array of zeros
        res[(resize_size[0]-rows)//2:(resize_size[0]-rows)//2+rows,
            (resize_size[1]-cols)//2:(resize_size[1]-cols)//2+cols,:] = img # fill in image in middle of zeros
                
        # Augmentation 
        if augment:            
            rows,cols = res.shape[:2]
            # calculates affine rotation with random angle rotation, keeping same center and scale
            M = cv.getRotationMatrix2D((cols/2,rows/2),np.random.uniform(0.0,360.0,1),1) 
            # applies affine rotation
            res = cv.warpAffine(res,M,(cols,rows))

        # Change to gray scale if input argument num_color_channels = 1
        if num_color_channels == 1: 
            res = cv.cvtColor(res, cv.COLOR_BGR2GRAY)# convert from bgr to gray
            res = res[...,None] # add extra dimension with blank values to very end, needed for keras
            
        batch_out[count%batch_size,...] = res # put image in position in batch, never to exceed size of batch
        
        for k in label_map.keys():
            if k in f: #if a category name is found in the path to the file of the image
                labels_out[count%batch_size,:] = np_utils.to_categorical(label_map[k],num_classes) #one hot labeling
                break   
                
        count += 1
        if count == num_files:# if gone through all files, restart the counter
            count = 0
        if count%batch_size == 0: #if gone through enough files to make a full batch
            if predict: # i.e., there is no label for this batch of images, so in prediction mode
                yield batch_out.astype(np.float)/255.
            else: # training
                yield batch_out.astype(np.float)/255., labels_out
            

### Pre-processing

In [4]:
# Get full paths to all classification data
# Data is assumed to reside under the directory "root_dir", and data for each class is assumed to reside in a separate subfolder
# root_dir = '/Users/dtaniguchi/Research/Image_classification/Scripps_plankton_camera_system_images/Practice_images'
root_dir = '/Users/dtaniguchi/Research/Image_classification/Scripps_plankton_camera_system_images/Labeled_ciliates_and_other'


img_types=['.jpg', '.tiff', '.tif', '.png', '.jpeg']

files = get_image_files(root_dir, img_types)
print('number of files is ',len(files))
print('example file names are ', files[0:4])

# Get the dimension range of the data for informational purposes
minh,minw,maxh,maxw = get_dimensions(files)
print('Over all images - minimum height: {}, minimum width: {}, maximum height: {}, maximum width:{}'.format(minh,minw,maxh,maxw))

# Assign numerical labels to categories - the number of categories is equal to the number of subfolders
label_map = make_labels(files)
print(label_map)

# Split the data into training and validation
train_files, val_files, train_labels, val_labels = make_train_val(files, label_map)
print(len(train_files))
print(len(val_files))

print('train labels length is ',len(train_labels))
print('validation labels length is', len(val_labels))
      
      

number of files is  1679
example file names are  ['/Users/dtaniguchi/Research/Image_classification/Scripps_plankton_camera_system_images/Labeled_ciliates_and_other/Ciliate/SPCP2-1551916275-018743-003-12-304-120-104.jpg', '/Users/dtaniguchi/Research/Image_classification/Scripps_plankton_camera_system_images/Labeled_ciliates_and_other/Ciliate/SPCP2-1549460119-039576-001-664-1800-96-104.jpg', '/Users/dtaniguchi/Research/Image_classification/Scripps_plankton_camera_system_images/Labeled_ciliates_and_other/Ciliate/SPCP2-1564409932-302705-002-192-932-104-96.jpg', '/Users/dtaniguchi/Research/Image_classification/Scripps_plankton_camera_system_images/Labeled_ciliates_and_other/Ciliate/SPCP2-1564466851-138834-002-792-708-136-144.jpg']
Over all images - minimum height: 24, minimum width: 32, maximum height: 312, maximum width:448
{'Ciliate': 0, 'Other': 1}
1008
671
train labels length is  1008
validation labels length is 671


### Fine tuning
The code below was taken from https://keras.io/applications/#fine-tune-inceptionv3-on-a-new-set-of-classes and must be adapted for use with xception instead of InceptionV3

#### Running InceptionV3 just to see if I can get it to work

In [5]:
from keras.applications.inception_v3 import InceptionV3#--[don't need if running Xception]
#from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

In [6]:
#input_shape taken from get_dimensions in Jupyter notebook Image_classification
# create the base pre-trained model

#base_model = Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(880,920,3), pooling=None)

base_model = InceptionV3(weights='imagenet', include_top=False) #--[don't need if using Xception]

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer -- let's say we have x classes--determined by len(label_map)
predictions = Dense(len(label_map), activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics = ['accuracy'])



Instructions for updating:
Colocations handled automatically by placer.


In [7]:
BS = 32
batch_gen = get_batches(train_files,label_map,batch_size=BS,resize_size=[224, 224],num_color_channels=3)
val_gen = get_batches(val_files,label_map,batch_size=BS,resize_size=[224, 224],num_color_channels=3)

In [ ]:
# Code in this cell taken from 
#https://www.pyimagesearch.com/2018/12/24/how-to-use-keras-fit-and-fit_generator-a-hands-on-tutorial/
# train the model on the new data for a few epochs
# initialize the number of epochs and batch size
EPOCHS = 1000
BS = 32

# construct the training image generator for data augmentation
# aug = ImageDataGenerator(rotation_range=180, zoom_range=0.15,
#     width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
#     horizontal_flip=True, fill_mode="nearest")

# train the network
# H = model.fit_generator(aug.flow(train_files, train_labels, batch_size=BS),
#         validation_data=(val_files, val_labels), steps_per_epoch=len(train_files) // BS,epochs=EPOCHS)


# class PlotCallbacks(keras.Callback):
#     def on_epoch_end(self, batch, logs={}):
#         print(logs)
#         return


model.fit_generator(batch_gen,validation_data=val_gen,validation_steps =4, steps_per_epoch=len(train_files) // BS,epochs=EPOCHS,
                   workers=1, use_multiprocessing=False, verbose =2)


Instructions for updating:
Use tf.cast instead.
Epoch 1/1000


In [ ]:

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True 

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers

model.fit_generator(batch_gen,validation_data=val_gen,validation_steps =4, steps_per_epoch=len(train_files) // BS,epochs=EPOCHS,
                   workers=1, use_multiprocessing=False, verbose = 1)

## Prediction

In [ ]:
## TO DO predict using test data, not validation data
predict_gen = get_batches(val_files,label_map,batch_size=1,resize_size=[224, 224],num_color_channels=3)
prediction = model.predict_generator(predict_gen,steps = len(val_files))

In [ ]:
def convert_to_class(prediction,label_map):
    predict_max = np.argmax(prediction,axis=1)#provides index of max value out of prediction classes
    predict_label = []
    for i in range(len(predict_max)):
        for k,v in label_map.items():
                if predict_max[i] == v:
                    predict_label.append(k)
    return predict_label    

def prop_correct(predict_label,actual_label):
    correct_class = []
    for i in range(len(predict_label)):
        if predict_label[i]==actual_label[i]:
            correct_class.append(1)
        else:
            correct_class.append(0)
    num_correct = sum(correct_class)
    proportion_correct = num_correct/len(predict_label)
    return proportion_correct

In [ ]:
prediction
print(label_map)
print(prediction[0:5])

In [ ]:
predict_class = convert_to_class(prediction,label_map)
print(predict_class[0:5])
print(prop_correct(predict_class,val_labels))

In [ ]:
testing = []
predict_max = np.argmax(prediction,axis=1)#provides index of max value out of prediction classes

for i in range(len(predict_max)):
    for k,v in label_map.items():
            if predict_max[i] == v:
                testing.append(k)
                
list_comp = [k for x in predict_max for k,v in label_map.items() if predict_max[x]==v]

In [ ]:
print(prediction[0:6])
print(predict_max[0:6])
print(testing[0:6])


print(list_comp[0:6])
print(label_map)
dododo = convert_to_class(prediction,label_map)
print(dododo[0:6])